In [ ]:
!curl ipecho.net/plain
! python -m pip install "pymongo[srv]"==3.11
! pip  install pymongo[srv,tls]
! pip install dnspython
!pip install pandas

34.42.133.151Requirement already satisfied: pymongo[srv]==3.11 in /usr/local/lib/python3.10/dist-packages (3.11.0)


In [ ]:
# @title
from pymongo.mongo_client import MongoClient
from google.colab import userdata

password = userdata.get('mongoDB_password')

uri = "mongodb+srv://PakkyNoi:" + password + "@workshop-bakery.dltqwqx.mongodb.net/?retryWrites=true&w=majority&appName=Workshop-Bakery"
# Create a new client and connect to the server
client = MongoClient(uri)
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!


In [ ]:
import pandas as pd

In [ ]:
BakeryDB = client.Bakery

CakeCollection = BakeryDB.cakes
CustomerCollection = BakeryDB.customer_details
OrderCollection = BakeryDB.order_records

In [ ]:
# get full cake lists

# allCakeList = [data for data in CakeCollection.find()]
allCakeList = list(CakeCollection.find())

allCakeDF = pd.DataFrame(allCakeList)

allCakeDF

,_id,name,shortDescription,description,image,ingredients,recipe,stock
0,64759d513beb40594cb859e9,Chocolate Cake,Chocolate cake is a cake flavored with melted ...,Chocolate cake is made with chocolate; it can ...,https://addapinch.com/wp-content/uploads/2020/...,"[flour, sugar, cocoa powder]",Preheat your oven to the specified temperature...,25
1,64759e4c3beb40594cb859ed,Cheese Cake,Cheesecake is a sweet dessert consisting of on...,Cheesecake is a sweet dessert consisting of on...,https://sallysbakingaddiction.com/wp-content/u...,"[graham cracker crumbs, sugar, eggs, butter, s...",Mix graham cracker crumbs and melted butter fo...,40
2,664758fe06cdc709b70778de,example,example,example,https://url_example.jpeg,"[A, B, C, D]",example,100


In [ ]:
allCustomerList = list(CustomerCollection.find())

allCustomerDF = pd.DataFrame(allCustomerList)

allCustomerDF

,_id,firstname,lastname,age,gender,telephone,email,street,villageNumber,city,postal-code
0,664627cdf94f8b7a6dc89b5e,example,example,18,Male,090,example@gmail.com,example,22/2,example,12345
1,664761bd06cdc709b70778df,Veggiee,Pak,18,Male,090,example@gmail.com,example,22/2,example,12345


In [140]:
#get order details
allOrderList = list(OrderCollection.find())

allOrderDF = pd.DataFrame(allOrderList)

# allOrderList
allOrderDF

,_id,firstname,lastname,vegan,menu,quantity,orderingDate,pickupDate
0,664755c706cdc709b70778dd,Veggiee,Pak,No,Chocolate Cake,0,2024-05-17,2024-05-24
1,6647735706cdc709b70778e0,Veggiee,Pak,No,Cheese Cake,0,2024-05-17,2024-05-31


In [141]:
#get customer details from the order
from bson import ObjectId

def get_customer_data_by_order_id(order_id, OrderCollection, CustomerCollection):
    # Find the order document in OrderCollection
    orderDocument = OrderCollection.find_one({'_id': ObjectId(order_id)})

    if not orderDocument:
        return pd.DataFrame()  # Return an empty DataFrame if the order is not found

    # Extract the first name and last name from the order document
    firstname = orderDocument.get('firstname')
    lastname = orderDocument.get('lastname')

    # Find the customer document in CustomerCollection using the first name and last name
    selectedCustomerData = list(CustomerCollection.find({
        '$and': [
            {'firstname': firstname},
            {'lastname': lastname}
        ]
    }))

    # Create a DataFrame from the selectedCustomerData
    selectedCustomerData = pd.DataFrame(selectedCustomerData)

    return selectedCustomerData

In [142]:
wanted_object_id = "664755c706cdc709b70778dd" #@param {type : "string"}

selectedCustomerData = get_customer_data_by_order_id(wanted_object_id, OrderCollection, CustomerCollection)

selectedCustomerData

,_id,firstname,lastname,age,gender,telephone,email,street,villageNumber,city,postal-code
0,664761bd06cdc709b70778df,Veggiee,Pak,18,Male,090,example@gmail.com,example,22/2,example,12345


In [143]:
# make baking plan

print("baking schedule based on pickup date")
bakingSchedule = pd.DataFrame(list(OrderCollection.aggregate([{"$sort":{"pickupDate":1}}])))

bakingSchedule

baking schedule based on pickup date


,_id,firstname,lastname,vegan,menu,quantity,orderingDate,pickupDate
0,664755c706cdc709b70778dd,Veggiee,Pak,No,Chocolate Cake,0,2024-05-17,2024-05-24
1,6647735706cdc709b70778e0,Veggiee,Pak,No,Cheese Cake,0,2024-05-17,2024-05-31
